In [2]:
!sudo apt-get install openjdk-8-jdk-headless -qq > /dev/null


In [3]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"


In [5]:
!wget -q https://archive.apache.org/dist/hadoop/core/hadoop-3.2.2/hadoop-3.2.2.tar.gz
!tar xf hadoop-3.2.2.tar.gz


In [4]:
!sudo pwd

/workspaces/projchat


In [5]:
os.environ["HADOOP_HOME"] = "/workspaces/projchat/hadoop-3.2.2"


In [8]:
!wget -q https://archive.apache.org/dist/spark/spark-3.1.2/spark-3.1.2-bin-hadoop3.2.tgz
!tar xf spark-3.1.2-bin-hadoop3.2.tgz


In [6]:
os.environ["SPARK_HOME"] = "/workspaces/projchat/spark-3.1.2-bin-hadoop3.2"


In [7]:
!pip install -q findspark



[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: python -m pip install --upgrade pip


In [11]:
!wget -q https://archive.apache.org/dist/hive/hive-3.1.2/apache-hive-3.1.2-bin.tar.gz
!tar xf apache-hive-3.1.2-bin.tar.gz


In [8]:
os.environ["HIVE_HOME"] = "/workspaces/projchat/apache-hive-3.1.2-bin"


In [9]:
import sys
sys.path.append(os.environ['HADOOP_HOME'] + "/bin")
sys.path.append(os.environ['SPARK_HOME'] + "/bin")
sys.path.append(os.environ['HIVE_HOME'] + "/bin")


In [10]:
import findspark
findspark.init()

from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("Hadoop, Spark and Hive on Colab") \
    .config("spark.sql.warehouse.dir", "/content/warehouse") \
    .config("hive.metastore.uris", "thrift://localhost:9083") \
    .enableHiveSupport() \
    .getOrCreate()

# Test the Spark setup
spark.range(5).show()


23/04/26 21:36:11 WARN Utils: Your hostname, codespaces-4cd64b resolves to a loopback address: 127.0.0.1; using 172.16.5.4 instead (on interface eth0)
23/04/26 21:36:11 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
23/04/26 21:36:11 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


+---+
| id|
+---+
|  0|
|  1|
|  2|
|  3|
|  4|
+---+



In [11]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("Spark SQL on Colab") \
    .config("spark.jars.packages", "org.apache.hadoop:hadoop-aws:3.2.0") \
    .getOrCreate()


In [12]:
from pyspark.sql import Row

data = [Row(name='John', age=25), Row(name='Alice', age=28), Row(name='Bob', age=23)]
sample_df = spark.createDataFrame(data)


In [13]:
sample_df.createOrReplaceTempView("test_table")


In [14]:
result = spark.sql("SELECT * FROM test_table")
result.show()

average_age = spark.sql("SELECT AVG(age) as avg_age FROM test_table")
average_age.show()


+-----+---+
| name|age|
+-----+---+
| John| 25|
|Alice| 28|
|  Bob| 23|
+-----+---+

+------------------+
|           avg_age|
+------------------+
|25.333333333333332|
+------------------+



In [15]:
!wget http://qwone.com/~jason/20Newsgroups/20news-18828.tar.gz
!tar -xzf 20news-18828.tar.gz
!ls -la 20_newsgroups


--2023-04-26 21:36:52--  http://qwone.com/~jason/20Newsgroups/20news-18828.tar.gz
Resolving qwone.com (qwone.com)... 173.48.205.131
Connecting to qwone.com (qwone.com)|173.48.205.131|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 14666916 (14M) [application/x-gzip]
Saving to: ‘20news-18828.tar.gz.2’

20news-18828.tar.gz  21%[===>                ]   3.05M   764KB/s    eta 17s    ^C
ls: cannot access '20_newsgroups': No such file or directory


In [16]:
import os
from pyspark.sql.functions import input_file_name

data_dir = "./20news-18828/"
data_files = [os.path.join(data_dir, f) for f in os.listdir(data_dir)]

raw_data = spark.read.text(data_files)
df = raw_data.withColumn("filename", input_file_name())


In [18]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("20 Newsgroups Example").getOrCreate()


In [19]:
newsgroups_rdd = spark.sparkContext.wholeTextFiles('20news-18828/*/*')


In [20]:
newsgroups_df = newsgroups_rdd.toDF(['path', 'text'])


In [22]:
newsgroups_df.show(20, truncate=False)


+--------------------------------------------------------+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [23]:
from pyspark.ml.feature import Tokenizer
from pyspark.sql.functions import explode, col

# Tokenize the text column
tokenizer = Tokenizer(inputCol="text", outputCol="words")
words_df = tokenizer.transform(newsgroups_df)

# Explode the words column into separate rows
exploded_df = words_df.select(explode(col("words")).alias("word"))

# Count the occurrence of each word
word_counts_df = exploded_df.groupBy("word").count().sort("count", ascending=False)

# Show the top 10 most frequent words
word_counts_df.show(10)


+----+-------+
|word|  count|
+----+-------+
|    |1451425|
| the| 232216|
|  to| 117223|
|  of| 105430|
|   a|  96905|
| and|  90789|
|  in|  75236|
|  is|  66665|
|   i|  63418|
|that|  59121|
+----+-------+
only showing top 10 rows



In [24]:
import re
from pyspark.sql.functions import explode
from pyspark.sql.functions import split

# remove any non-words and split lines into separate words
# finally, convert all words to lowercase
def splitter(line):
    line = re.sub(r'^\W+|\W+$', '', line)
    return map(str.lower, re.split(r'\W+', line))

# create a Spark session
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("20 Newsgroups Example").getOrCreate()

# read the dataset into a DataFrame
newsgroups_df = spark.read.text('20news-18828/*/*')

# split the text into separate words
words_df = newsgroups_df.select(explode(split(newsgroups_df.value, r'\s+')).alias('word'))

# count the occurrences of each word
word_counts_df = words_df.groupBy('word').count()

# find the word with the highest count
max_word_count = word_counts_df.agg({'count': 'max'}).collect()[0]['max(count)']
max_word = word_counts_df.filter(word_counts_df['count'] == max_word_count).collect()[0]['word']

print("max: %s = %d" % (max_word, max_word_count))


max:  = 336257


DATA TRANSFORMATION

In [34]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import split, lower, length
from pyspark.ml.feature import Tokenizer, StopWordsRemover

spark = SparkSession.builder.appName("20 Newsgroups Example").getOrCreate()

newsgroups_rdd = spark.sparkContext.wholeTextFiles('20news-18828/*/*')
newsgroups_df = newsgroups_rdd.toDF(['path', 'text'])
newsgroups_df = newsgroups_df.select('path', split('path', '/').getItem(7).alias('category'), split('path', '/').getItem(8).alias('subcategory'), split('path', '/').getItem(9).alias('filename'), 'text')
tokenizer = Tokenizer(inputCol='text', outputCol='words')
newsgroups_df = newsgroups_df.withColumn('text', lower('text'))
newsgroups_df = tokenizer.transform(newsgroups_df)
remover = StopWordsRemover(inputCol='words', outputCol='filtered_words')
newsgroups_df = remover.transform(newsgroups_df)
newsgroups_df = newsgroups_df.withColumn('text_length', length('text'))


In [36]:
from pyspark.sql.functions import count
category_counts = newsgroups_df.groupBy('category').agg(count('*').alias('category_count')).orderBy('category_count', ascending=False)
subcategory_counts = newsgroups_df.groupBy('category', 'subcategory').agg(count('*').alias('subcategory_count')).orderBy('subcategory_count', ascending=False)


In [37]:
category_counts.show(10)
subcategory_counts.show(20)


+--------+--------------+
|category|category_count|
+--------+--------------+
|    null|         18828|
+--------+--------------+



+--------+-----------+-----------------+
|category|subcategory|subcategory_count|
+--------+-----------+-----------------+
|    null|       null|            18828|
+--------+-----------+-----------------+



In [38]:
from pyspark.sql.functions import avg, max
category_text_length = newsgroups_df.groupBy('category').agg(avg('text_length').alias('avg_text_length'), max('text_length').alias('max_text_length')).orderBy('avg_text_length', ascending=False)


In [39]:
category_text_length.show()


+--------+-----------------+---------------+
|category|  avg_text_length|max_text_length|
+--------+-----------------+---------------+
|    null|1801.127788400255|         160546|
+--------+-----------------+---------------+



In [48]:
from pyspark.sql.types import StringType, StructType, StructField

# Read data and retain the path information
rdd = spark.sparkContext.wholeTextFiles("20news-18828/*/*")

# Create a DataFrame with columns 'path', 'text', and 'label'
data = rdd.toDF(schema=StructType([
    StructField("path", StringType(), True),
    StructField("text", StringType(), True)
]))

# Extract labels from the file paths
from pyspark.sql.functions import regexp_extract

data = data.withColumn("label", regexp_extract("path", r".*\/(\w+\.*\w*)\/\d+", 1))


In [49]:
from pyspark.ml.feature import Tokenizer, StopWordsRemover, RegexTokenizer
from pyspark.sql.functions import regexp_replace

# Remove non-alphanumeric characters
data = data.withColumn("text", regexp_replace("text", "[^a-zA-Z0-9\s]", ""))

# Tokenize the text
tokenizer = RegexTokenizer(inputCol="text", outputCol="words", pattern="\\W")
data = tokenizer.transform(data)

# Remove stop words
stopwords_remover = StopWordsRemover(inputCol="words", outputCol="filtered_words")
data = stopwords_remover.transform(data)


In [50]:
from pyspark.ml.feature import StringIndexer

label_indexer = StringIndexer(inputCol="label", outputCol="labelIndex")
data = label_indexer.fit(data).transform(data)


In [51]:
train_data, test_data = data.randomSplit([0.8, 0.2], seed=42)


In [52]:
from pyspark.ml.feature import HashingTF, IDF
from pyspark.ml.classification import LogisticRegression
from pyspark.ml import Pipeline

# Feature extraction with HashingTF and IDF
hashingTF = HashingTF(inputCol="filtered_words", outputCol="rawFeatures")
idf = IDF(inputCol="rawFeatures", outputCol="features")

# Classifier
lr = LogisticRegression(maxIter=10, regParam=0.001, labelCol="labelIndex")

# Build the pipeline
pipeline = Pipeline(stages=[hashingTF, idf, lr])


In [53]:
# Train the model
model = pipeline.fit(train_data)

# Make predictions on the test data
predictions = model.transform(test_data)


23/04/26 21:57:06 WARN DAGScheduler: Broadcasting large task binary with size 4.1 MiB
23/04/26 21:57:12 WARN DAGScheduler: Broadcasting large task binary with size 4.1 MiB
23/04/26 21:57:18 WARN BLAS: Failed to load implementation from: com.github.fommil.netlib.NativeSystemBLAS
23/04/26 21:57:18 WARN BLAS: Failed to load implementation from: com.github.fommil.netlib.NativeRefBLAS
23/04/26 21:57:20 WARN DAGScheduler: Broadcasting large task binary with size 4.1 MiB
23/04/26 21:57:23 WARN DAGScheduler: Broadcasting large task binary with size 4.1 MiB
23/04/26 21:57:26 WARN DAGScheduler: Broadcasting large task binary with size 4.1 MiB
23/04/26 21:57:30 WARN DAGScheduler: Broadcasting large task binary with size 4.1 MiB
23/04/26 21:57:36 WARN DAGScheduler: Broadcasting large task binary with size 4.1 MiB
23/04/26 21:57:42 WARN DAGScheduler: Broadcasting large task binary with size 4.1 MiB
23/04/26 21:57:51 WARN DAGScheduler: Broadcasting large task binary with size 4.1 MiB
23/04/26 21:58:

Py4JJavaError: An error occurred while calling o703.fit.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 40.0 failed 1 times, most recent failure: Lost task 0.0 in stage 40.0 (TID 22429) (181bbee3-9c22-4d60-a188-1dd1c1f8992c.internal.cloudapp.net executor driver): java.lang.OutOfMemoryError: Java heap space
	at java.util.Arrays.copyOf(Arrays.java:3236)
	at java.io.ByteArrayOutputStream.grow(ByteArrayOutputStream.java:118)
	at java.io.ByteArrayOutputStream.ensureCapacity(ByteArrayOutputStream.java:93)
	at java.io.ByteArrayOutputStream.write(ByteArrayOutputStream.java:153)
	at org.apache.spark.util.ByteBufferOutputStream.write(ByteBufferOutputStream.scala:41)
	at java.io.ObjectOutputStream$BlockDataOutputStream.write(ObjectOutputStream.java:1853)
	at java.io.ObjectOutputStream.write(ObjectOutputStream.java:709)
	at org.apache.spark.util.Utils$.writeByteBuffer(Utils.scala:233)
	at org.apache.spark.scheduler.DirectTaskResult.$anonfun$writeExternal$1(Tas

----------------------------------------
Exception occurred during processing of request from ('127.0.0.1', 52560)
ERROR:root:Exception while sending command.
Traceback (most recent call last):
  File "/workspaces/projchat/spark-3.1.2-bin-hadoop3.2/python/lib/py4j-0.10.9-src.zip/py4j/java_gateway.py", line 1207, in send_command
    raise Py4JNetworkError("Answer from Java side is empty")
py4j.protocol.Py4JNetworkError: Answer from Java side is empty

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/workspaces/projchat/spark-3.1.2-bin-hadoop3.2/python/lib/py4j-0.10.9-src.zip/py4j/java_gateway.py", line 1033, in send_command
    response = connection.send_command(command)
  File "/workspaces/projchat/spark-3.1.2-bin-hadoop3.2/python/lib/py4j-0.10.9-src.zip/py4j/java_gateway.py", line 1211, in send_command
    raise Py4JNetworkError(
py4j.protocol.Py4JNetworkError: Error while receiving
Traceback (most recent call last):
  Fi

In [ ]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

evaluator = MulticlassClassificationEvaluator(labelCol="labelIndex", predictionCol="prediction", metricName="accuracy")
accuracy = evaluator.evaluate(predictions)
print("Test set accuracy = {:.2f}".format(accuracy))
